# Pythagoras Trees

In [1]:
"""
A notebook to produce the openSCAD instructions to draw pythagoras trees
"""

from math import degrees, radians, sin, asin, cos, sqrt, pi
from queue import SimpleQueue
from decimal import Decimal
from random import randint

In [2]:
# Global variables
# a list of pythagorean triplets that can be chose in producing tree leave
triplets     = [(3,4,5),(5,12,13),(8,15,17),(20,21,29),(28,45,53),(33,56,65),(48,55,73),(36,77,85),(39,80,89),(65,72,97)] # (12,35,37),(7,24,25),(16,63,65),(9,40,41),(11,60,61),(13,84,85)
# All triplets up to 100. Above triplets have been filtered so that sides are not so similar in size.
# [(3,4,5),(5,12,13),(8,15,17),(7,24,25),(20,21,29),(12,35,37),(9,40,41),(28,45,53),(11,60,61),(16,63,65),(33,56,65),(48,55,73),(13,84,85),(36,77,85),(39,80,89),(65,72,97)]


In [7]:
def draw_leaf(leaf, q, max_order=4):
    """ 
    Return the openSCAD instructions to draw a leaf of a pythagoras tree

    Parameters:

    leaf:   A dictionary containing
                "order":                    # to control iterations 
                "triplet":                  # pythagorean triplet to use in the construction of the tree
                "gamma":                    # angle, use pi/2 = 90 deg for pythagorean and pi/3 = 60 deg for equal sides (1,1,1)
                "origin":                   # origin coordinates for a leaf
                "rotation"                  # rotation of base tree square
                "side":                     # size of tree square's side
                "alternate":                # wether to alternate the triangle legs, boolean
                "random":                   # wether to use random triplets from global `triplets` list, boolean
           
    q:          a queue of all the leaves that have to be drawn
    
    max_order:  the maximum iteration
    
    
    Returns:

    A string with openSCAD instructions to draw a leaf of a pythagoras tree.
    Instructions of all leaves must be concatenated to draw the tree.
    """

    ins   = []
    triplet = leaf["triplet"]
    a,b,c = triplet
    gamma = leaf["gamma"]
    x,y   = leaf["origin"]
    rot   = leaf["rotation"]
    sz    = leaf["side"]
    order = leaf["order"]
    alternate = leaf["alternate"]
    random = leaf["random"]
    if order > max_order: 
        return ""
    ins.append(f"translate([{x}, {y}, 0])")
    ins.append(f"rotate([0, 0, {degrees(rot)}])")
    ins.append(f"cuboid([{sz},{sz},5], center=false, fillet=0.3);")
    # leaf a
    beta = asin(b*sin(gamma)/c)
    la = { "order"   : order+1, 
           "triplet" : triplet, 
           "gamma"   : gamma,
           "origin"  : (x + sz*cos(pi/2+rot), y + sz*sin(pi/2+rot)),
           "rotation": rot + beta,
           "side"    : sz * a / c,
           "alternate": alternate, "random": random}
    # leaf b
    xa, ya = la["origin"]
    sza = la["side"]
    ins.append(f"translate([{xa},{ya},0]) cylinder(h=1, d={sza/100});")
    xc, yc = (xa + sz*cos(rot), ya + sz*sin(rot))
    ins.append(f"translate([{xc},{yc},0]) cylinder(h=1, d={sza/100});")
    alfa = asin(a*sin(gamma)/c)
    lb = { "order"   : order+1, 
           "triplet" : triplet, 
           "gamma"   : gamma,
           "origin"  : (xa + sza*cos(rot+beta), ya + sza*sin(rot+beta)),
           "rotation": rot - alfa,
           "side"    : sz * b / c,
           "alternate": alternate, "random": random}
    xb, yb = lb["origin"]
    szb = lb["side"]
    ins.append(f"translate([{xb},{yb},0]) cylinder(h=1, d={sza/100});")
    if random == True:
        #trp = triplets[randint(0,len(triplets)-1)]
        la["triplet"] = triplets[randint(0,len(triplets)-1)] #trp
        lb["triplet"] = triplets[randint(0,len(triplets)-1)] #trp
    if alternate == True and order%2 == 0:
        la["triplet"] = (b,a,c)
        lb["triplet"] = (b,a,c)
    q.put(la)
    q.put(lb)
    return " ".join(ins)

In [8]:
def draw_tree(q, max_order=2):
    instructions = []
    ins = "\n"
    while ins != "":
        ins = draw_leaf(q.get(), q, max_order=max_order)
        instructions.append(ins)                      
    return "\n".join(instructions)

In [9]:
# Example

leafs = SimpleQueue()
leafs.put({ "order": 0,                 # to control iterations 
            "triplet":(3,4,5),          # pythagorean triplet to use in the construction of the tree
            "gamma":pi/2,               # angle, use pi/2 = 90 deg for pythagorean and pi/3 = 60 deg for equal sides (1,1,1)
            "origin":(0.0, 0.0),        # origin coordinates for a leaf
            "rotation":radians(0),      # rotation of base tree square
            "side":20,                  # size of tree square's side
            "alternate":True,           # wether to alternate the triangle legs
            "random":False              # wether to use random triplets from global `triplets` list
})

print(draw_tree(leafs))

translate([0.0, 0.0, 0]) rotate([0, 0, 0.0]) cuboid([20,20,5], center=false, fillet=0.3); translate([1.2246467991473533e-15,20.0,0]) cylinder(h=1, d=0.12); translate([20.0,20.0,0]) cylinder(h=1, d=0.12); translate([7.2,29.6,0]) cylinder(h=1, d=0.12);
translate([1.2246467991473533e-15, 20.0, 0]) rotate([0, 0, 53.13010235415599]) cuboid([12.0,12.0,5], center=false, fillet=0.3); translate([-9.6,27.2,0]) cylinder(h=1, d=0.096); translate([-2.4000000000000004,36.8,0]) cylinder(h=1, d=0.096); translate([-9.6,36.8,0]) cylinder(h=1, d=0.096);
translate([7.2, 29.6, 0]) rotate([0, 0, -36.86989764584402]) cuboid([16.0,16.0,5], center=false, fillet=0.3); translate([16.8,42.4,0]) cylinder(h=1, d=0.128); translate([29.6,32.8,0]) cylinder(h=1, d=0.128); translate([29.6,42.4,0]) cylinder(h=1, d=0.128);
translate([-9.6, 27.2, 0]) rotate([0, 0, 90.0]) cuboid([9.6,9.6,5], center=false, fillet=0.3); translate([-19.2,27.2,0]) cylinder(h=1, d=0.0768); translate([-19.2,36.8,0]) cylinder(h=1, d=0.0768); trans